### Student Information 
Name: 林靖淵<br>
Student ID: 113356040<br>
GitHub ID: https://github.com/jing-yuan-nccu<br>
Kaggle name: jingyaun_nccu<br>
Kaggle private scoreboard snapshot: <br>
***
### Instructions
1. First: This part is worth 30% of your grade. Do the take home exercises in the DM2024-Lab2-master Repo. You may need to copy some cells from the Lab notebook to this notebook.
2. Second: This part is worth 30% of your grade. Participate in the in-class Kaggle Competition regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place
in the Private Leaderboard ranking:
Bottom 40%: Get 20% of the 30% available for this section.
Top 41% - 100%: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)
Submit your last submission BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday). Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the img folder of this repository and rerun the cell Student Information.
3. Third: This part is worth 30% of your grade. A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model.
You can also mention different things you tried and insights you gained.
4. Fourth: This part is worth 10% of your grade. It's hard for us to follow if your code is messy :'(, so please tidy up your notebook.
Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.
Make sure to commit and save your changes to your repository BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)

## First Part

In [ ]:
import pandas as pd

### training data
anger_train = pd.read_csv("data/semeval/train/anger-ratings-0to1.train.txt",
                         sep="\t", header=None,names=["id", "text", "emotion", "intensity"])
sadness_train = pd.read_csv("data/semeval/train/sadness-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
fear_train = pd.read_csv("data/semeval/train/fear-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
joy_train = pd.read_csv("data/semeval/train/joy-ratings-0to1.train.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
# combine 4 sub-dataset
train_df = pd.concat([anger_train, fear_train, joy_train, sadness_train], ignore_index=True)

In [ ]:
### testing data
anger_test = pd.read_csv("data/semeval/dev/anger-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
sadness_test = pd.read_csv("data/semeval/dev/sadness-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
fear_test = pd.read_csv("data/semeval/dev/fear-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])
joy_test = pd.read_csv("data/semeval/dev/joy-ratings-0to1.dev.gold.txt",
                         sep="\t", header=None, names=["id", "text", "emotion", "intensity"])

# combine 4 sub-dataset
test_df = pd.concat([anger_test, fear_test, joy_test, sadness_test], ignore_index=True)

---
### ** >>> Exercise 1 (Take home): **  
Plot word frequency for Top 30 words in both train and test dataset. (Hint: refer to DM lab 1)

In [ ]:
# Answer
import nltk
import helpers.data_mining_helpers as dmh

# do word seperation
X_df = pd.concat([train_df, test_df], ignore_index=True)
nltk.download('punkt_tab')
X_df['unigrams'] = X_df['text'].apply(lambda x: dmh.tokenize_text(x))

# convert each unigram to vertor form
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(X_df.text) #learn the vocabulary and return document-term matrix
print(X_counts.shape)

# concate terms and frenquencies together in a dataframe
plot_x = ["term_"+str(i) for i in count_vect.get_feature_names_out()[:]]
plot_y = ["id_"+ str(i) for i in list(X_df.index)[:]]
plot_z = X_counts[:, :].toarray() #X_counts[how many documents, how many terms]

df_todraw = pd.DataFrame(plot_z, columns = plot_x, index = plot_y)

# sum and sort
df_sum = df_todraw.sum(axis=0)
df_sum_sorted = df_sum.sort_values(ascending=False)

In [ ]:
import plotly.express as px

# plot top 30 words frequency 
df_plot = pd.DataFrame({
    'Term': df_sum_sorted.index[:30],      # Term names as index
    'Frequency': df_sum_sorted.values[:30] # Frequencies as values
})

fig = px.bar(df_plot, x="Term", y='Frequency',color="Frequency",  # Assign color based on frequency values
             color_continuous_scale="Viridis")

# Update layout (rotating labels, sizing, etc.)
fig.update_layout(
    xaxis_tickangle=90,  # Rotate x-axis labels
    height=500,  # Set figure height
    width=1200,  # Set figure width
    title="Term Frequencies",
    xaxis_title="Terms",
    yaxis_title="Frequency"
)

# Show the figure
fig.show()

---
### ** >>> Exercise 2 (Take home): **  
Generate an embedding using the TF-IDF vectorizer instead of th BOW one with 1000 features and show the feature names for features [100:110].

In [ ]:
# Answer here
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,  # Limit the number of features (optional)
    # stop_words='english',  # Remove stopwords (optional)
    ngram_range=(1, 1)  # Use unigrams only; adjust for bigrams/trigrams if needed
)

# Fit and transform the corpus to generate TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(train_df['text'])

# Convert the TF-IDF matrix to a DataFrame for better readability
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),  # Convert sparse matrix to dense array
    columns=tfidf_vectorizer.get_feature_names_out()  # Feature names (words)
)

In [ ]:
feature_names_1000 = tfidf_vectorizer.get_feature_names_out()
feature_names_1000[100:110]

---
### ** >>> Exercise 3 (Take home): **  
Can you interpret the results above? What do they mean?

***Answer***<br>
The diagonal values (57, 76, 56, 47) represent the number of instances correctly predicted for each emotion, while the other numbers indicate the misclassified instances where the column emotion was mistakenly predicted as the row emotion. From the confusion matrix, it can be observed that the emotion "fear" is more prone to being misclassified as other emotions. <br>Notably, among the four emotions, there is a higher rate of mutual misclassification between "anger" and "fear." This could be due to the difficulty in distinguishing between these two emotions, as well as between "fear" and "sadness." It is also possible that the emotion "fear" sometimes exhibits characteristics of anger or sadness, leading to these misclassifications.

---
### ** >>> Exercise 4 (Take home): **  
Build a model using a ```Naive Bayes``` model and train it. What are the testing results? 

*Reference*: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [ ]:
# Answer here
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load your dataset (example uses pandas DataFrame)
import pandas as pd

## build DecisionTree model
DT_model = MultinomialNB()

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

In [ ]:
# Confusion matrix
## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

print(classification_report(y_true=y_test, y_pred=y_test_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

---
### ** >>> Exercise 5 (Take home): **  

How do the results from the Naive Bayes model and the Decision Tree model compare? How do you interpret these differences? Use the theoretical background covered in class to try and explain these differences.

***Answer***<br>
Based on the comparison, it can be observed that Naive Bayes is relatively weaker in distinguishing angry emotions. Additionally, there is some similarity between fear and sadness or anger, leading to occasional misclassifications. More prominent errors (i.e., a higher number of misclassifications) are present across all emotional categories except for joy. Overall, Naive Bayes is generally recognized to produce better results because it excels at handling high-dimensional textual data. However, its performance in distinguishing anger and sadness is weaker, possibly due to the assumption of feature independence in Naive Bayes. Emotions like anger and sadness require the interdependence of features for more accurate classification. On the other hand, fear-related emotions tend to achieve better results under the assumption of feature independence.

---
### ** >>> Exercise 6 (Take home): **  

Plot the Training and Validation Accuracy and Loss (different plots), just like the images below.(Note: the pictures below are an example from a different model). How to interpret the graphs you got? How are they related to the concept of overfitting/underfitting covered in class?

In [ ]:
# Answer here
df = pd.DataFrame(training_log)
plt.figure(figsize=(14, 6))
# Subplot 1: Accuracy
plt.subplot(1, 2, 1)
plt.plot(df['epoch'], df['accuracy'], label='Accuracy', marker='o')
plt.plot(df['epoch'], df['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()

# Subplot 2: Loss
plt.subplot(1, 2, 2)
plt.plot(df['epoch'], df['loss'], label='Loss', marker='o')
plt.plot(df['epoch'], df['val_loss'], label='Validation Loss', marker='o')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# Show the plots
plt.tight_layout()
plt.show()

***Answer***<br>
Observing the Model Accuracy and Model Loss plots can help identify when the model starts to overfit. From the Accuracy plot, it can be seen that before epoch 5, the growth trend of the validation accuracy begins to plateau. At this point, only the training accuracy continues to increase, while the validation accuracy remains stable. This indicates that the model starts to overfit around epoch 5. Similarly, in the Model Loss plot, the validation loss starts to increase sharply at epoch 5, whereas the training loss continues to decrease. This is another clear sign of overfitting.

---
### ** >>> Exercise 7 (Take home): **  

Now, we have the word vectors, but our input data is a sequence of words (or say sentence). 
How can we utilize these "word" vectors to represent the sentence data and train our model?


***Answer***<br>
We can calculate the average of words in a sentence

In [ ]:
def sentence_vector(sentence, model):
    # Filter out words not in the model's vocabulary
    words = [word for word in sentence if word in model.wv] 
    if not words:
        return np.zeros(model.vector_size)
    # Compute the mean of the word vectors
    return np.mean(model.wv[words], axis=0)

train_df["sv"] = train_df["text_tokenized"].apply(lambda x: sentence_vector(x,word2vec_model))
train_df["sv"].shape

---
### ** >>> Exercise 8 (Take home): **  

Generate a t-SNE and UMAP visualization to show the 15 words most related to the words "angry", "happy", "sad", "fear" (60 words total). Compare the differences between both graphs.

In [ ]:
# Answer here
# Data preparation
word_list = ['angry', 'happy', 'sad', 'fear']

topn = 15
angry_words = ['angry'] + [word_ for word_, sim_ in w2v_google_model.most_similar('angry', topn=topn)]        
happy_words = ['happy'] + [word_ for word_, sim_ in w2v_google_model.most_similar('happy', topn=topn)]
sad_words = ['sad'] + [word_ for word_, sim_ in w2v_google_model.most_similar('sad', topn=topn)]        
fear_words = ['fear'] + [word_ for word_, sim_ in w2v_google_model.most_similar('fear', topn=topn)]        

target_words = angry_words + happy_words + sad_words + fear_words
cn = topn + 1
color = ['r'] * cn + ['y'] * cn + ['b'] * cn + ['g'] * cn

from sklearn.manifold import TSNE
import umap.umap_ as umap

## w2v model
model = w2v_google_model

## prepare training word vectors
size = 200
target_size = len(target_words)
all_word = list(model.index_to_key)
word_train = target_words + all_word[:size]
X_train = model[word_train]

## UMAP model
umap_model = umap.UMAP(n_components=2, metric='cosine', random_state=28)

## training
X_umap = umap_model.fit_transform(X_train)

## t-SNE model
tsne = TSNE(n_components=2, metric='cosine', random_state=28)

## training
X_tsne = tsne.fit_transform(X_train)

In [ ]:
# Plot the result with two subplots in one canvas
fig, axes = plt.subplots(1, 2, figsize=(15, 7.5), dpi=115)

# First subplot
axes[0].scatter(X_tsne[:target_size, 0], X_tsne[:target_size, 1], c=color)
axes[0].set_title('Plot 1: t-SNE Visualization')
for label, x, y in zip(target_words, X_tsne[:target_size, 0], X_tsne[:target_size, 1]):
    axes[0].annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')

# Second subplot (can use different data or the same for demonstration)
axes[1].scatter(X_umap[:target_size, 0], X_umap[:target_size, 1], c=color)
axes[1].set_title('Plot 2: UMAP Visualization with Labels')
for label, x, y in zip(target_words, X_umap[:target_size, 0], X_umap[:target_size, 1]):
    axes[1].annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')

# Adjust layout
plt.tight_layout()
plt.show()

***Answer***<br>
Observing the two plots, it can be seen that the UMAP results are more compact, while the t-SNE results are more dispersed. If all points are displayed in the same color, UMAP's results make it easier to distinguish which points form a group compared to t-SNE. This indirectly suggests that t-SNE may overlook global relationships.

### ** >>> Exercise 9 (Take home): **  

You noticed there is a **role** parameter inside the ollama.chat function, investigate what other roles there can be inside the function and what do they do. Give an example of a prompt using another role in additional to the **user** one.

***Answer***<br>
There are three roles can be inside the function. The other two roles are "system" and "assistant".

In [ ]:
response = ollama.chat(
            model='llava-phi3',
            messages=[{
                'role': 'system',
                'content': 'Summarize this text. Chroma is a vector database that stores the document embeddings.It creates a vector space from the text chunks (splits) and their corresponding embeddings (embeddings).This vector space allows for similarity-based comparisons, enabling efficient document retrieval.'
            }]
        )
print(response['message']['content'])

### ** >>> Exercise 10 (Take home): **  

Try asking the model with one image of your choosing. Is the description accurate? Why?

In [ ]:
# Answer here
response4 = ollama.chat(model='llava-phi3', messages=[
    {
        'role': 'user',
        'content': 'What is this image about?',
        'images': ['./pics/example3.jpg'] #Image with the cat
    },
])

display(Markdown(response4['message']['content']))

***Answer***<br>
To be honest, the description accurate is good. It's is far from my expection. I think it is because I give it a simple picture with only one objects. Also, from the response, we can tell the model trainer gave the generation a structure to follow. Maybe that why the description is good.


### ** >>> Exercise 11 (Take home): **  

Try to modify the code to make it accept **three URLs**, or **three text documents** of your choosing. After modifying it, make **three prompts/questions** with information that can be found in each of the documents/urls, **compare the accuracy of the response** with the actual answer. Investigate and discuss the advantages and disadvantages of RAG systems.

In [ ]:
# Answer here
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

# Function to load, split, and retrieve documents from multiple URLs
def load_and_retrieve_multiple_docs(urls):
    all_splits = []  # To store chunks from all URLs

    # Loop through each URL
    for url in urls:
        loader = WebBaseLoader(
            web_paths=(url,),
            bs_kwargs=dict()
        )
        docs = loader.load()  # Load documents from the URL
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(docs)  # Split into chunks
        all_splits.extend(splits)  # Add chunks to the combined list

    # Generate embeddings and create vector store
    embeddings = OllamaEmbeddings(model=llm_model)
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

    return vectorstore.as_retriever()

# List of URLs to process
urls = [
    "https://www.ibm.com/topics/large-language-models",
    "https://www.ibm.com/topics/data-mining",
    "https://www.ibm.com/topics/chatbots"
]

# Create the retriever
retriever = load_and_retrieve_multiple_docs(urls)


In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) #Format the retrieved docs in an orderly manner for prompting

# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3.2', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

questions = [
    "What is LLMs?",
    "What is data mining?",
    "What is chatbot?"
]
# Use the RAG chain
for q in questions:
    result = rag_chain(q)
    display(Markdown(result))

***Answer***<br>
RAG has strong scalability, allowing the model to connect to external resources and provide responses that are more up-to-date. However, this adds additional processes, which on one hand, consume computational resources, and on the other hand, may lead to a decline in response quality if there are issues in the RAG process. For example, retrieving incorrect documents can negatively impact the final output.

### ** >>> Exercise 12 (Take home): **

Follow Exercise 6 again and Plot the Training and Validation Accuracy and Loss for the results of this Neural Network. Compare the results of both KNN and the NN we just implemented. Discuss about why we obtained these results with the LLM Embeddings compared to the results of the other models implemented in this Lab.

In [ ]:
# Answer here
df = pd.DataFrame(training_log)
plt.figure(figsize=(14, 6))
# Subplot 1: Accuracy
plt.subplot(1, 2, 1)
plt.plot(df['epoch'], df['accuracy'], label='Accuracy', marker='o')
plt.plot(df['epoch'], df['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()

# Subplot 2: Loss
plt.subplot(1, 2, 2)
plt.plot(df['epoch'], df['loss'], label='Loss', marker='o')
plt.plot(df['epoch'], df['val_loss'], label='Validation Loss', marker='o')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# Show the plots
plt.tight_layout()
plt.show()

***Answer***<br>
It can be observed that the results using embeddings are worse than those from the previous model. Whether using a neural network or the KNN method, the performance with embeddings is inferior to the previous model. This might indicate that embeddings are not well-suited for emotion detection, or that the sentences may require more preprocessing to help the model capture the key information in the text.

### ** >>> Exercise 13 (Take home): **

Compare and discuss the results of the zero-shot, 1-shot and 5-shot classification.

***Answer***<br>
It can be observed that among 0-shot, 1-shot, and 5-shot scenarios, 1-shot performs the best. Compared to 0-shot, providing an example in the prompt indeed helps the LLM better understand the task at hand. However, excessive prompting does not seem to yield better results.

## Third

In the Kaggle competition, I tried several methods. For embedding, I primarily used TF-IDF, though I also attempted using Word2Vec, but the process took too much time. Regarding model selection, I experimented with three machine learning models: Random Forest, Decision Tree, and XGBoost. Additionally, I also tried using deep learning methods.

### Machine Learning

In [ ]:
import json
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

***Data login & processing***

In [ ]:
data = []
with open('dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
 
f.close()

In [ ]:
emotion = pd.read_csv('dm-2024-isa-5810-lab-2-homework/emotion.csv')
data_identification = pd.read_csv('dm-2024-isa-5810-lab-2-homework/data_identification.csv')

In [ ]:
df = pd.DataFrame(data)
_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})
df = df.merge(data_identification, on='tweet_id', how='left')

train_data = df[df['identification'] == 'train']
test_data = df[df['identification'] == 'test']

In [ ]:
train_data = train_data.merge(emotion, on='tweet_id', how='left')

In [ ]:
train_data.drop_duplicates(subset=['text'], keep=False, inplace=True)

In [ ]:
train_data.head()

***Prepare data for training model***<br>
Due to the large size of the dataset, I chose to sample a certain percentage of the data for training. Even so, using just 20% of the data still required about an hour of training. Once I found suitable parameter combinations, I used the entire dataset for training. While this approach is not entirely rigorous, it serves as a useful reference.

In [ ]:
train_data_sample = train_data.sample(frac=1, random_state=42)

In [ ]:
y_train_data = train_data['emotion']
X_train_data = train_data.drop(['tweet_id', 'emotion', 'identification', 'hashtags'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_data, y_train_data, test_size=0.2, random_state=42, stratify=y_train_data)

In [ ]:
tfidf = TfidfVectorizer(max_features=500)
X = tfidf.fit_transform(X_train['text']).toarray()
X_test = tfidf.transform(X_test['text'])

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
label_mapping = dict(zip(le.classes_, range(len(le.classes_))))

***Training model & Accuracy***<br>
For the machine learning model selection, I tried XGBoost, Random Forest, and Decision Tree. Among them, XGBoost performed very poorly, producing outputs limited to only three emotion categories, resulting in poor outcomes. Initially, I determined that this was not a data issue, as the input was consistent across all models. Therefore, I decided not to use XGBoost as my model. As for Decision Tree and Random Forest, while Decision Tree trained faster, Random Forest produced better results. Ultimately, I chose to fine-tune Random Forest to find the best-performing model.

In [ ]:
from xgboost import XGBClassifier

# Initialize the XGBoost classifier
clf = XGBClassifier()
# Fit the classifier to your data
clf.fit(X, y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini', random_state=42)
clf.fit(X, y)

In [ ]:
# Set parameters
param_grid = {
    'n_estimators': 100,
    'max_depth': 20,
    'min_samples_split': 5,
    'min_samples_leaf': 3
}

# Train the RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X, y)

***Try different combination of parameters***<br>
I tried a total of five parameters for Random Forest, each with three different values, resulting in 3^5 combinations. Ultimately, I found that the combination of max_depth=20 and max_features="sqrt" performed the best. However, the accuracy was around 0.44 for all combinations. The primary goal was to find a suitable set of parameters to avoid overfitting caused by a lack of parameter control.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from joblib import Parallel, delayed
from tqdm import tqdm

# Parameter grid
param_grid = {
    'n_estimators': [50],
    'max_depth': [20],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['sqrt']
}

# Wrap the grid search process with tqdm
def tqdm_grid_search(cv, estimator, param_grid, X, y, scoring='accuracy'):
    param_list = list(ParameterGrid(param_grid))  # Generate all parameter combinations
    results = []
    for params in tqdm(param_list, desc="Grid Search Progress"):
        clf = estimator.set_params(**params)
        scores = []
        for train_idx, test_idx in cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            clf.fit(X_train, y_train)
            scores.append(clf.score(X_test, y_test))
        results.append((params, scores))
    return results

# Example usage
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=3)

# Call tqdm_grid_search
results = tqdm_grid_search(
    cv=cv,
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    X=np.array(X),  # Convert to numpy array if needed
    y=np.array(y),  # Convert to numpy array if needed
)

# Print results
print("Best Parameters and Scores:")
for params, scores in results:
    print(f"Params: {params}, Mean Accuracy: {np.mean(scores):.4f}")


In [ ]:
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))
print(accuracy_score(y, y_pred_train))

In [ ]:
from sklearn.metrics import classification_report
# Generate classification report
report = classification_report(y_test, y_pred, target_names=le.classes_, digits=4)

In [ ]:
print(report)

### Deep learning 
I also tried using deep learning methods to train the model. Although the results were better than Random Forest, the training time was significantly longer. Therefore, I decided to first focus on fine-tuning the Random Forest model before adjusting the deep learning approach.

In [ ]:
import keras
from sklearn.preprocessing import LabelEncoder

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

In [ ]:
# I/O check
input_shape = X.shape[1]
print('input_shape: ', input_shape)

output_shape = len(le.classes_)
print('output_shape: ', output_shape)

In [ ]:
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
x = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(x)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

In [ ]:
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 4
batch_size = 32

# training!
history = model.fit(X, y, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')

In [ ]:
#Let's take a look at the training log
training_log = pd.DataFrame()
training_log = pd.read_csv("logs/training_log.csv")
training_log

In [ ]:
# Answer here
df = pd.DataFrame(training_log)
plt.figure(figsize=(14, 6))
# Subplot 1: Accuracy
plt.subplot(1, 2, 1)
plt.plot(df['epoch'], df['accuracy'], label='Accuracy', marker='o')
plt.plot(df['epoch'], df['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()

# Subplot 2: Loss
plt.subplot(1, 2, 2)
plt.plot(df['epoch'], df['loss'], label='Loss', marker='o')
plt.plot(df['epoch'], df['val_loss'], label='Validation Loss', marker='o')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# Show the plots
plt.tight_layout()
plt.show()

### Use OpenAI embedding
I originally planned to use OpenAI's API for embedding, but I found that it required a lot of time—processing 300K records would take over 3 hours. Therefore, I determined that this method was not suitable for my current situation.

In [ ]:
# Example data (replace with your actual data)
X_train_texts = X_train['text'].tolist()  # Convert training text column to a list
X_test_texts = X_test['text'].tolist()    # Convert testing text column to a list

In [ ]:
import openai
import numpy as np
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm  # For showing progress bars
import getpass
import os

# Set OpenAI API Key
API_Key = ""
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

# Function to generate embeddings from OpenAI
def generate_embeddings(texts, model="text-embedding-ada-002"):
    embeddings = []
    for text in tqdm(texts, desc="Generating embeddings"):
        response = client.embeddings.create(input=text, model=model)
        embedding = response.data[0].embedding
        embeddings.append(embedding)
    return np.array(embeddings)

# Generate embeddings for train and test data
train_embeddings = generate_embeddings(X_train_texts)
test_embeddings = generate_embeddings(X_test_texts)

# Reduce dimensions to 500 using Truncated SVD
'''svd = TruncatedSVD(n_components=500, random_state=42)
train_embeddings_500 = svd.fit_transform(train_embeddings)
test_embeddings_500 = svd.transform(test_embeddings)

# Train embeddings are now reduced to 500 dimensions
print("Shape of Train Embeddings:", train_embeddings_500.shape)
print("Shape of Test Embeddings:", test_embeddings_500.shape)'''


### Generate Answer

In [ ]:
X_test_data = test_data.drop(['tweet_id', 'identification', 'hashtags'], axis=1)

In [ ]:
X_test_data = tfidf.transform(X_test_data['text']).toarray()

In [ ]:
# deep learning
y_test_pred = model.predict(X_test_data, batch_size=128)
y_pred_labels = label_decode(label_encoder, y_test_pred)

In [ ]:
y_pred_labels

In [ ]:
# machine learning
y_test_pred = clf.predict(X_test_data)

In [ ]:
y_pred_labels = le.inverse_transform(y_test_pred)
y_pred_labels

In [ ]:
submission = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred_labels
})

In [ ]:
submission.to_csv('kaggle/submission.csv')